In [1]:
import torch

In [2]:
torch.cuda.is_available()

False

In [3]:
torch.backends.mps.is_available()

True

In [4]:
tensor0d = torch.tensor(1.0)
tensor1d = torch.tensor([1.0, 2.0, 3.0])
tensor2d = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
tensor3d = torch.tensor([[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], [[7.0, 8.0, 9.0], [10.0, 11.0, 12.0]]])

print(tensor0d)
print(tensor1d)
print(tensor2d)
print(tensor3d)

tensor(1.)
tensor([1., 2., 3.])
tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor([[[ 1.,  2.,  3.],
         [ 4.,  5.,  6.]],

        [[ 7.,  8.,  9.],
         [10., 11., 12.]]])


In [5]:
tensor2d = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
print(tensor2d.T)

tensor([[1., 4.],
        [2., 5.],
        [3., 6.]])


In [6]:
print(tensor2d @ tensor2d.T)

tensor([[14., 32.],
        [32., 77.]])


In [7]:
import torch.nn.functional as F

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2])
b = torch.tensor([0.0])
z = w1 * x1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)

In [8]:
import torch.nn.functional as F

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)
z = w1 * x1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)

grad_L_w1 = torch.autograd.grad(loss, w1, retain_graph=True)
grad_L_b = torch.autograd.grad(loss, b, retain_graph=True)

print(grad_L_w1)
print(grad_L_b)

loss.backward()
print(w1.grad)
print(b.grad)

(tensor([-0.0898]),)
(tensor([-0.0817]),)
tensor([-0.0898])
tensor([-0.0817])


In [9]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_imputs, num_outputs):
        super().__init__()
        self.layers = torch.nn.Sequential(
            # Input layer
            torch.nn.Linear(num_imputs, 30),
            torch.nn.ReLU(),

            # Hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # Output layer
            torch.nn.Linear(20, num_outputs)
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

model = NeuralNetwork(num_imputs=50, num_outputs=3)
print(model)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params} trainable parameters.')

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)
The model has 2213 trainable parameters.


In [10]:
print(model.layers[0].weight)
print(model.layers[0].weight.shape)

Parameter containing:
tensor([[-0.1206,  0.0056, -0.0019,  ..., -0.1105,  0.0031,  0.0791],
        [ 0.1088,  0.0236, -0.0300,  ...,  0.0368, -0.0485, -0.1176],
        [ 0.0677, -0.1007, -0.1073,  ..., -0.0326,  0.0698,  0.0072],
        ...,
        [-0.1252,  0.0005,  0.0852,  ...,  0.0588, -0.0789, -0.0345],
        [ 0.0097, -0.1151, -0.0979,  ...,  0.1291,  0.1091,  0.0596],
        [ 0.1070, -0.0769,  0.0145,  ...,  0.0326, -0.0863,  0.0437]],
       requires_grad=True)
torch.Size([30, 50])


In [11]:
torch.manual_seed(123)
x = torch.rand((1, 50))
out = model(x)
print(out)

tensor([[-0.0403,  0.3371,  0.1414]], grad_fn=<AddmmBackward0>)


In [ ]:
# 推論だけの場合
with torch.no_grad():
    out = model(x)
    print(out)

tensor([[-0.0403,  0.3371,  0.1414]])


In [ ]:
# 推論だけの場合
with torch.no_grad():
    out = torch.softmax(model(x), dim=1)
    print(out)

tensor([[0.2734, 0.3987, 0.3279]])


In [16]:
X_train = torch.tensor([[-1.2, 3.1], [-0.9, 2.9], [-0.5, 2.6], [2.3, -1.1], [2.7, -1.5]])
y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([[-0.8, 2.8], [2.6, -1.6]])
y_test = torch.tensor([0, 1])

In [17]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, index):
        x = self.features[index]
        y = self.labels[index]
        return x, y

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [18]:
print(len(train_ds))

5


In [21]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [22]:
for idx, (x, y) in enumerate(train_loader):
    print(f'Batch {idx + 1}')
    print('x:', x)
    print('y:', y)


Batch 1
x: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]])
y: tensor([1, 0])
Batch 2
x: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]])
y: tensor([0, 0])


In [ ]:
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_imputs=2, num_outputs=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5) 

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params} trainable parameters.')

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch + 1}, Batch {batch_idx + 1}, Loss: {loss.item():.4f}')

The model has 752 trainable parameters.
Epoch 1, Batch 1, Loss: 0.7487
Epoch 1, Batch 2, Loss: 0.6450
Epoch 2, Batch 1, Loss: 0.4423
Epoch 2, Batch 2, Loss: 0.1256
Epoch 3, Batch 1, Loss: 0.0269
Epoch 3, Batch 2, Loss: 0.0043
tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [26]:
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [27]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])


In [28]:
predictions = torch.argmax(probas, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [30]:
def compute_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total_examples = 0
    
    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            outputs = model(features)
        predicted = torch.argmax(outputs, dim=1)
        compare = (predicted == labels)
        correct += torch.sum(compare)
        total_examples += len(compare)
    return (correct / total_examples).item()

print('Train accuracy:', compute_accuracy(model, train_loader))

Train accuracy: 1.0


In [31]:
torch.save(model.state_dict(), 'toy_model.pth')

In [32]:
model = NeuralNetwork(num_imputs=2, num_outputs=2)
model.load_state_dict(torch.load('toy_model.pth'))

<All keys matched successfully>

In [33]:
torch.manual_seed(123)
model = NeuralNetwork(num_imputs=2, num_outputs=2)

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        features, labels = features.to(device), labels.to(device)
        logits = model(features)
        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch + 1}, Batch {batch_idx + 1}, Loss: {loss.item():.4f}')

Epoch 1, Batch 1, Loss: 0.7487
Epoch 1, Batch 2, Loss: 0.6450
Epoch 2, Batch 1, Loss: 0.4423
Epoch 2, Batch 2, Loss: 0.1256
Epoch 3, Batch 1, Loss: 0.0269
Epoch 3, Batch 2, Loss: 0.0043
